In [ ]:
!pip install tabula-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 5.2 MB/s 


In [ ]:
import tabula
import pandas as pd
import os
from google.colab import files

In [ ]:
df = tabula.read_pdf('/content/20220101freshsales_report.pdf',pages='all')

In [ ]:
df_1 = tabula.read_pdf('/content/20220201freshsales_report.pdf',pages='all')
df[0]=pd.concat([df_1[0],df[0]],axis=0)

In [ ]:
df[0].head()

,SalesAccount : id,Account : Name,Account : Parent\raccount,Account : Products,Account : Academy\ruser,Account : Number of\ruser,Account : Subscription\rend date,Account : Client code
0,5000267315,Aman Resorts and Hotels,NaN,"Infor DEPM,SunSystems",Yes,250.0,30-06-2022,27DA025
1,5000272144,PAVILIONS RESORTS LIMITED,NaN,"Other,SunSystems",Yes,3.0,28-02-2022,27DP013
2,5002715373,Eastin Hanoi Hotel,Absolute Hotel Services,"SunSystems,TRG Products",Yes,4.0,12-09-2022,131E021
0,5000267315,Aman Resorts and Hotels,NaN,"Infor DEPM,SunSystems",Yes,250.0,30-06-2022,27DA025
1,5002715373,Eastin Hanoi Hotel,Absolute Hotel Services,"SunSystems,TRG Products",Yes,4.0,12-09-2022,131E021


In [ ]:
df[0].columns

Index(['SalesAccount : id', 'Account : Name', 'Account : Parent\raccount',
       'Account : Products', 'Account : Academy\ruser',
       'Account : Number of\ruser', 'Account : Subscription\rend date',
       'Account : Client code'],
      dtype='object')

In [ ]:
def convert_date_time(Str):
  dd=Str[0:2]
  MM=Str[3:5]
  yyyy=Str[6:10]
  Str=yyyy+'-'+MM+'-'+dd
  return Str
listi=[]
for i in (df[0]['Account : Subscription\rend date']):
  listi.append(convert_date_time(i))
df[0]['Account : Subscription\rend date']=listi

In [ ]:
import datetime
tday=datetime.date.today()
tdelta=datetime.timedelta(days=180)
pd.to_datetime(tday,format='%Y-%m-%d')

Timestamp('2022-08-04 00:00:00')

In [ ]:
df[0][df[0].columns[6]]=pd.to_datetime(df[0][df[0].columns[6]],format='%Y-%m-%d')

In [ ]:
df[0]['Subscription end date in 6 months']=[1]*(len(df[0][df[0].columns[6]]))

In [ ]:
listi=[]
for i in df[0][df[0].columns[6]]:
  if (tday-tdelta) < i and (tday+tdelta) > i:
    listi.append(1)
  else:
    listi.append(0)
df[0]['Subscription end date in 6 months']=listi

In [ ]:
df[0]['Subscription end date in 6 months']

0    1
1    1
2    1
0    1
1    1
Name: Subscription end date in 6 months, dtype: int64

In [ ]:
filtDate=(df[0]['Subscription end date in 6 months']==1)
df_filtDate=df[0].loc[filtDate]
df_filtDate

,SalesAccount : id,Account : Name,Account : Parent\raccount,Account : Products,Account : Academy\ruser,Account : Number of\ruser,Account : Subscription\rend date,Account : Client code,Subscription end date in 6 months
0,5000267315,Aman Resorts and Hotels,NaN,"Infor DEPM,SunSystems",Yes,250.0,2022-06-30,27DA025,1
1,5000272144,PAVILIONS RESORTS LIMITED,NaN,"Other,SunSystems",Yes,3.0,2022-02-28,27DP013,1
2,5002715373,Eastin Hanoi Hotel,Absolute Hotel Services,"SunSystems,TRG Products",Yes,4.0,2022-09-12,131E021,1
0,5000267315,Aman Resorts and Hotels,NaN,"Infor DEPM,SunSystems",Yes,250.0,2022-06-30,27DA025,1
1,5002715373,Eastin Hanoi Hotel,Absolute Hotel Services,"SunSystems,TRG Products",Yes,4.0,2022-09-12,131E021,1


In [ ]:
df[0]['SalesAccount : id'].value_counts()

5000267315    2
5002715373    2
5000272144    1
Name: SalesAccount : id, dtype: int64

In [ ]:
dfCountSalesaccountId=df[0].groupby('SalesAccount : id').size().reset_index(name='Count')

In [ ]:
listNewSalesAccountId=(dfCountSalesaccountId['Count']==1)
listNewSalesAccountId=list(dfCountSalesaccountId.loc[listNewSalesAccountId]['SalesAccount : id'])
listNewSalesAccountId

[5000272144]

In [ ]:
filtNewSalesAccountId=df[0]['SalesAccount : id'].isin(listNewSalesAccountId)
df_filtNewSalesAccountId=df[0].loc[filtNewSalesAccountId]

In [ ]:
df[0].to_excel('output_1.xlsx')
df_filtDate.to_excel('output_2.xlsx')
df[0].loc[filtNewSalesAccountId].to_excel('output_3.xlsx')
